<font color=red size=6>**Sistemas Atómicos**

<font color=red>**Prof. Enrique Mejía Ospino, emejia@uis.edu.co**

<font color=red>**Escuela de Química**
    
<font color=red>**Universidad Industrial de Santander** 

**<font color=blue> Vamos  atilizar el modulo *Pyscf* para desarrollar algunos ejercicios de cálculo mecánico-cuántico sobre sistemas atómicos.** 

In [2]:
from ase.units import Ry, eV, Ha
import pyscf
import h5py
from pyscf import gto
from pyscf.data import elements
from pyscf.scf import atom_hf
import numpy as np
import matplotlib.pyplot as plt
import fortecubeview # Visualizar moléculas

**<font color=blue> En el primer ejemplo se muestra cálculos MC utilizando base ECP a átomos con números atómicos entre 20-93.** 

In [7]:
# RHF
atoms = {}
for z in range(10, 94):
    try:
        a = gto.M(atom=[[z, (0, 0, 0)]], basis='lanl2dz', ecp='lanl2dz', verbose=0, spin=None)
        atoms[z] = a
    except:
        pass

def count_scf_cycle(envs):
    envs['mf']._cycle = envs['cycle']

counts = {}
for z, atm in atoms.items():
    mf = atom_hf.AtomSphericAverageRHF(atm)
    mf.atomic_configuration = elements.NRSRHF_CONFIGURATION
    mf.callback = count_scf_cycle
    mf.run()

    mf1 = atom_hf.AtomSphericAverageRHF(atm)
    mf1.atomic_configuration = elements.CONFIGURATION
    mf1.callback = count_scf_cycle
    mf1.run()
    counts[z] = (mf._cycle, mf.e_tot, mf1._cycle, mf1.e_tot)

print('E(NRSRHF) is lower in %d samples' % sum([(v[1] - v[3] < 1e-7) for v in counts.values()]))
print('E(NRSRHF) is higher in %d samples' % sum([(v[1] - v[3] > 1e-7) for v in counts.values()]))

ECP lanl2dz not found for  Ne


E(NRSRHF) is lower in 58 samples
E(NRSRHF) is higher in 4 samples


In [8]:
counts

{10: (4, -128.52235440171017, 4, -128.52235440171023),
 11: (3, -0.12780949381299156, 3, -0.12780949381299156),
 12: (3, -0.781026619756843, 3, -0.781026619756843),
 13: (4, -1.768592563721537, 4, -1.7685925637215374),
 14: (4, -3.448742049228529, 4, -3.448742049228529),
 15: (4, -5.970055536734266, 4, -5.9700555367342645),
 16: (4, -9.570036794300366, 4, -9.570036794300368),
 17: (4, -14.483601404494992, 4, -14.483601404494992),
 18: (4, -20.67320487408709, 4, -20.673204874087087),
 19: (4, -27.80957412158366, 4, -27.809574121583662),
 20: (5, -36.21397107570574, 5, -36.21397107570574),
 21: (5, -45.711328074082374, 5, -45.74307367349415),
 22: (6, -57.01377412624966, 6, -57.01377412624966),
 23: (6, -69.93082525847663, 6, -69.93082525847666),
 24: (6, -84.62017858827954, 6, -84.58405264797685),
 25: (8, -102.14184046234114, 6, -102.04243615919137),
 26: (7, -121.71789761573997, 6, -121.46868401128216),
 27: (7, -143.51809919835594, 6, -143.11077681912477),
 28: (7, -168.0365711427935

In [14]:
import os
import time
import pyscf
from pyscf.tools.mo_mapping import mo_comps

log = pyscf.lib.logger.Logger(verbose=5)
with open('/proc/cpuinfo') as f:
    for line in f:
        if 'model name' in line:
            log.note(line[:-1])
            break
with open('/proc/meminfo') as f:
    log.note(f.readline()[:-1])
log.note('OMP_NUM_THREADS=%s\n', os.environ.get('OMP_NUM_THREADS', None))

for bas in ('3-21g', '6-31g*', 'cc-pVTZ', 'ANO-Roos-TZ'):
    mol = pyscf.M(atom = 'N 0 0 0; N 0 0 1.1',
                  basis = bas)
    cpu0 = time.clock(), time.time()

    mf = mol.RHF().run()
    cpu0 = log.timer('N2 %s RHF'%bas, *cpu0)

    mymp2 = mf.MP2().run()
    cpu0 = log.timer('N2 %s MP2'%bas, *cpu0)

    mymc = mf.CASSCF(4, 4)
    idx_2pz = mo_comps('2p[xy]', mol, mf.mo_coeff).argsort()[-4:]
    mo = mymc.sort_mo(idx_2pz, base=0)
    mymc.kernel(mo)
    cpu0 = log.timer('N2 %s CASSCF'%bas, *cpu0)

    mycc = mf.CCSD().run()
    cpu0 = log.timer('N2 %s CCSD'%bas, *cpu0)

    mf = mol.RKS().run(xc='b3lyp')
    cpu0 = log.timer('N2 %s B3LYP'%bas, *cpu0)

    mf = mf.density_fit().run()
    cpu0 = log.timer('N2 %s density-fit RHF'%bas, *cpu0)


model name	: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz
MemTotal:        8277196 kB
OMP_NUM_THREADS=None

converged SCF energy = -108.300041526319
    CPU time for N2 3-21g RHF      0.62 sec, wall time      0.09 sec
E(MP2) = -108.537199201249  E_corr = -0.237157674930381
    CPU time for N2 3-21g MP2      0.00 sec, wall time      0.00 sec
CASSCF energy = -108.399344825887
CASCI E = -108.399344825887  E(CI) = -5.6974097910152  S^2 = 0.0000000
    CPU time for N2 3-21g CASSCF      1.23 sec, wall time      0.16 sec
E(CCSD) = -108.5269196185218  E_corr = -0.2268780922030085
    CPU time for N2 3-21g CCSD      1.58 sec, wall time      0.21 sec
converged SCF energy = -108.839820000931
    CPU time for N2 3-21g B3LYP      2.08 sec, wall time      0.30 sec
converged SCF energy = -108.839874858427
    CPU time for N2 3-21g density-fit RHF      2.23 sec, wall time      0.29 sec
converged SCF energy = -108.94154785202
    CPU time for N2 6-31g* RHF      0.59 sec, wall time      0.10 sec
E(MP2) = -1

**<font color=blue> Similar al anterior script.** 

In [16]:
from pyscf import gto, scf, dft, tddft

mol = gto.Mole()
mol.build(
    atom = '''
      o     0    0       0
      h     0    -.757   .587
      h     0    .757    .587''',  # in Angstrom
    basis = '631g',
    symmetry = True,
)

mf = dft.RKS(mol)
mf.xc = 'b3lyp'
mf.kernel() # Calcula energía del estado fundamental 

mytd = mf.TDDFT().run() # Calcula los estados excitados 

mytd.nstates = 10
mytd.kernel()
mytd.analyze()

converged SCF energy = -76.347810766126
Excited State energies (eV)
[7.80914054 9.91154603 9.94779907]
Excited State energies (eV)
[ 7.80914054  9.91154601  9.94779907 12.37308116 14.75039315 18.18007294
 27.76941228 28.14881774 29.14720826 30.09284831]

** Singlet excitation energies and oscillator strengths **
Excited State   1:   B2      7.80914 eV    158.77 nm  f=0.0115
Excited State   2:   A1      9.91155 eV    125.09 nm  f=0.0962
Excited State   3:   A2      9.94780 eV    124.63 nm  f=0.0000
Excited State   4:   B1     12.37308 eV    100.20 nm  f=0.0870
Excited State   5:   B1     14.75039 eV     84.05 nm  f=0.4120
Excited State   6:   A1     18.18007 eV     68.20 nm  f=0.2413
Excited State   7:   A2     27.76941 eV     44.65 nm  f=0.0000
Excited State   8:   A1     28.14882 eV     44.05 nm  f=0.0167
Excited State   9:   B2     29.14721 eV     42.54 nm  f=0.0869
Excited State  10:   B1     30.09285 eV     41.20 nm  f=0.1475


In [6]:
from pyscf import gto, scf
import pyscf
from pyscf.geomopt.geometric_solver import optimize

mol = gto.Mole()
mol.build(
    atom = '''
      o     0    0       0
      h     0    -.757   .587
      h     0    .757    .587''',  # in Angstrom
    basis = 'ccpvdz',
    symmetry = True,
)

mf = scf.RHF(mol)

#
# geometry optimization for HF.  There are two entries to invoke the geomeTRIC
# optimization
#
# method 1: import the optimize function from pyscf.geomopt.geometric_solver
mol_eq = pyscf.optimize(mf)
print(mol_eq.atom_coords())

# method 2: create the optimizer from Gradients class
mol_eq = mf.Gradients().optimizer(solver='geomeTRIC').kernel()

#
# geometry optimization for CASSCF
#
from pyscf import mcscf
mf = scf.RHF(mol)
mc = mcscf.CASSCF(mf, 4, 4)
conv_params = {
    'convergence_energy': 1e-4,  # Eh
    'convergence_grms': 3e-3,    # Eh/Bohr
    'convergence_gmax': 4.5e-3,  # Eh/Bohr
    'convergence_drms': 1.2e-2,  # Angstrom
    'convergence_dmax': 1.8e-2,  # Angstrom
}
# method 1
mol_eq = optimize(mc, **conv_params)

# method 2
mol_eq = mc.Gradients().optimizer(solver='geomeTRIC').kernel(conv_params)

AttributeError: module 'pyscf' has no attribute 'optimize'

**<font color=blue> Similar al anterior script, un poco más sencillo, se utiliza la determinación de la energía del estado fundamental usando RHF.** 

In [7]:
mol = gto.Mole()
mol.build(
    atom = '''
      o     0    0       0
      h     0    -.757   .587
      h     0    .757    .587''',  # in Angstrom
    basis = '631g',
    symmetry = True,
)
mytd = mol.RHF().run().TDHF().run() # Calcula energía del estado fundamental usando el métod RHF y los estados excitados en
# una sola línea

#mytd = mf.TDDFT().run() # Calcula los estados excitados 

mytd.nstates = 10
mytd.kernel()
mytd.analyze()

converged SCF energy = -75.9839484981055
Excited State energies (eV)
[ 9.36273381 11.28221712 11.78434553]
Excited State energies (eV)
[ 9.36273381 11.28221712 11.78434554 13.85939254 15.47459866 19.10080506
 30.71639446 30.80765858 31.41780461 32.07833016]

** Singlet excitation energies and oscillator strengths **
Excited State   1:   B2      9.36273 eV    132.42 nm  f=0.0145
Excited State   2:   A2     11.28222 eV    109.89 nm  f=0.0000
Excited State   3:   A1     11.78435 eV    105.21 nm  f=0.1126
Excited State   4:   B1     13.85939 eV     89.46 nm  f=0.0970
Excited State   5:   B1     15.47460 eV     80.12 nm  f=0.4419
Excited State   6:   A1     19.10081 eV     64.91 nm  f=0.2685
Excited State   7:   B2     30.71639 eV     40.36 nm  f=0.0641
Excited State   8:   A2     30.80766 eV     40.24 nm  f=0.0000
Excited State   9:   A1     31.41780 eV     39.46 nm  f=0.0106
Excited State  10:   B2     32.07833 eV     38.65 nm  f=0.0045


In [8]:
from pyscf import gto, scf, dft, tddft

mol = gto.Mole()
mol.build(
    atom = '''
      o     0    0       0
      h     0    -.757   .587
      h     0    .757    .587''',  # in Angstrom
    basis = '631g',
    symmetry = True,
)

mf = mol.RKS()
mf.xc= 'camb3lyp'
mf.run()

# Note you need to switch to xcfun library for cam-b3lyp tddft
mf._numint.libxc = pyscf.dft.xcfun
mytd = mf.TDDFT()

mytd.nstates = 10
mytd.kernel()
mytd.analyze()



converged SCF energy = -76.3554935000355
Excited State energies (eV)
[ 7.90669935 10.01488148 10.08746356 12.53394723 14.79181987 18.32040498
 27.99990594 28.40056132 29.31788336 30.37368889]

** Singlet excitation energies and oscillator strengths **
Excited State   1:   B2      7.90670 eV    156.81 nm  f=0.0113
Excited State   2:   A1     10.01488 eV    123.80 nm  f=0.0943
Excited State   3:   A2     10.08746 eV    122.91 nm  f=0.0000
Excited State   4:   B1     12.53395 eV     98.92 nm  f=0.0833
Excited State   5:   B1     14.79182 eV     83.82 nm  f=0.4130
Excited State   6:   A1     18.32040 eV     67.68 nm  f=0.2443
Excited State   7:   A2     27.99991 eV     44.28 nm  f=0.0000
Excited State   8:   A1     28.40056 eV     43.66 nm  f=0.0145
Excited State   9:   B2     29.31788 eV     42.29 nm  f=0.0827
Excited State  10:   B1     30.37369 eV     40.82 nm  f=0.1574


**<font color=blue> Aqui usamos CIS para determinar los estados excitados con amplitudes TDA.** 

In [9]:
from pyscf import gto, scf, dft, tddft

mol = gto.Mole()
mol.build(
    atom = '''
      o     0    0       0
      h     0    -.757   .587
      h     0    .757    .587''',  # in Angstrom
    basis = '631g',
    symmetry = True,
)

mf = dft.RKS(mol)
mf.xc = 'b3lyp'
mf.kernel()

mytd = tdscf.TDA(mf).run(nstates=5)
mytd.analyze()

def tda_denisty_matrix(td, state_id):
    '''
    Taking the TDA amplitudes as the CIS coefficients, calculate the density
    matrix (in AO basis) of the excited states
    '''
    cis_t1 = td.xy[state_id][0]
    dm_oo =-np.einsum('ia,ka->ik', cis_t1.conj(), cis_t1)
    dm_vv = np.einsum('ia,ic->ac', cis_t1, cis_t1.conj())

    # The ground state density matrix in mo_basis
    mf = td._scf
    dm = np.diag(mf.mo_occ)

    # Add CIS contribution
    nocc = cis_t1.shape[0]
    # Note that dm_oo and dm_vv correspond to spin-up contribution. "*2" to
    # include the spin-down contribution
    dm[:nocc,:nocc] += dm_oo * 2
    dm[nocc:,nocc:] += dm_vv * 2

    # Transform density matrix to AO basis
    mo = mf.mo_coeff
    dm = np.einsum('pi,ij,qj->pq', mo, dm, mo.conj())
    return dm

# Density matrix for the 3rd excited state
dm = tda_denisty_matrix(mytd, 2)

# Write to cube format
from pyscf.tools import cubegen
cubegen.density(mol, 'tda_density.cube', dm)

# Write the difference between excited state and ground state
cubegen.density(mol, 'density_diff.cube', dm-mf.make_rdm1())

# The positive and negative parts can be overlayed in Jmol
# isosurface ID "surf1" cutoff  0.02 density_diff.cube
# isosurface ID "surf2" cutoff -0.02 density_diff.cube

converged SCF energy = -76.3478107661261


NameError: name 'tdscf' is not defined

**<font color=blue> Comparando dos funcionales.** 

In [10]:
import copy
from pyscf import gto, dft, tddft

mol = gto.M(atom = '''
      o     0    0       0
      h     0    -.757   .587
      h     0    .757    .587''',  # in Angstrom
    basis = '631g*',
    symmetry = True,
)


mf = dft.RKS(mol).run(xc='pbe0')

#
# A common change for TDDFT is to use different XC functional library.  For
# example, PBE0 is not supported by the default XC library (libxc) in the TDDFT
# calculation.  Changing to xcfun library for TDDFT can solve this problem
#
mf._numint.libxc = dft.xcfun
# PySCF-1.6.1 and newer supports the .TDDFT method to create a TDDFT
# object after importing tdscf module.
td = mf.TDDFT()
print(td.kernel()[0] * 27.2114)

#
# Overwriting the relevant attributes of the ground state mf object,
# the TDDFT calculations can be run with different XC, grids.
#
mf.xc = 'lda,vwn'
mf.grids.set(level=2).kernel(with_non0tab=True)
td = mf.TDDFT()
print(td.kernel()[0] * 27.2114)

#
# Overwriting the ground state SCF object is unsafe.  A better solution is to
# create a new fake SCF object to hold different XC, grids parameters.
#
from pyscf.dft import numint
mf = dft.RKS(mol).run(xc='pbe0')
mf1 = copy.copy(mf)
mf1.xc = 'lda,vwn'
mf1.grids = dft.Grids(mol)
mf1.grids.level = 2
mf1._numint = numint.NumInt()
mf1._numint.libxc = dft.xcfun
td = mf1.TDDFT()
print(td.kernel()[0] * 27.2114)


converged SCF energy = -76.3238441273012
Excited State energies (eV)
[ 8.40219954 10.40170929 10.92823644]
[ 8.40220385 10.40171464 10.92824206]
Excited State energies (eV)
[10.97713746 13.11294174 13.34875981]
[10.9771431  13.11294848 13.34876667]
converged SCF energy = -76.3238441273012
Excited State energies (eV)
[10.97713746 13.11294174 13.34875981]
[10.9771431  13.11294848 13.34876667]
